### Assignment 4

#### 1. Consider the two-way random effects model given in (6.13) of Ch.6 lecture notes:The regressors matrix X can contain time-invariant and individual-invariant regressors. The two-way random effects estimation follows the following iterative algorithm: ...

##### (a) Write a Python script that implements the above algorithm, and apply it to estimate. The output should be in a table form containing the following columns of results: parameter estimates, standard errors, tstatistics and p-values. The table should also have row and column names.

In [1]:
### Load the data and create Z_state and Z_yr
import numpy as np
import pandas as pd

df = pd.read_excel('cigar.xlsx')
df.columns = df.columns.str.strip()
df = df.dropna()
### These dummies are used to create the Q matrix
Z_STATE = pd.get_dummies(df['STATE'])
Z_YR = pd.get_dummies(df['YR'])

### This dummy are used to include in X, we drop first in order to avoid multicollinearity
Z_YR_1 = pd.get_dummies(df['YR'],drop_first=True)

In [2]:
### Choose the dependent variable and the independent variables
Y = np.log(df['C'])
df['ln_Price'] = np.log(df['Price'])
df['ln_NDI'] = np.log(df['NDI'])
df['ln_PIMIN'] = np.log(df['PIMIN'])
X = df[['ln_Price','ln_NDI','ln_PIMIN']]

In [3]:
### Get the X matrix; we drop the constant and only drop the first dummy in Year dummy
Random_X = pd.concat([X,Z_STATE,Z_YR_1],axis=1)
Random_X = np.array(Random_X,dtype=float)
Y = np.array(Y,dtype=float)

In [4]:
### Prepare for the previous prepared data

# Get the beta and the residual
beta = np.linalg.inv(Random_X.T @ Random_X) @ Random_X.T @ Y
residual = Y - Random_X @ beta


### Estimate the variance of the u_hat and update
def update_the_variance_matrix(residual,Z_STATE,Z_YR):
    ##compute the w_hat
    def get_w_hat(Q,u_hat):
        numerator = u_hat.T @ Q @ u_hat
        denominator = np.trace(Q)
        return numerator/denominator
    
    ## Preapre for the matrix for computing the variance matrix
    Z_STATE_Z_STATE = Z_STATE @ Z_STATE.T
    Z_YR_Z_YR = Z_YR @ Z_YR.T

    J_n = np.ones(Z_STATE.shape[1]).reshape(Z_STATE.shape[1],1)@np.ones(Z_STATE.shape[1]).reshape(Z_STATE.shape[1],1).T
    J_t = np.ones(Z_YR.shape[1]).reshape(Z_YR.shape[1],1)@np.ones(Z_YR.shape[1]).reshape(Z_YR.shape[1],1).T


    bar_J_n = (1/(Z_STATE.shape[1])) * J_n
    bar_J_t = (1/(Z_STATE.shape[1])) * J_t


    E_n = np.eye(Z_STATE.shape[1]) - bar_J_n
    E_t = np.eye(Z_YR.shape[1]) - bar_J_t

    ### estimate the Q matrix
    Q_1 = np.kron(E_n,E_t)
    Q_2 = np.kron(E_n,bar_J_t)
    Q_3 = np.kron(bar_J_n,E_t)
    Q_4 = np.kron(bar_J_n,bar_J_t)

    #estimate the w_hat
    w_1 = get_w_hat(Q_1,residual)
    w_2 = get_w_hat(Q_2,residual)
    w_3 = get_w_hat(Q_3,residual)
    w_4 = get_w_hat(Q_4,residual)


    #compute the variance matrix
    variance_matrix = w_1*Q_1 + w_2*Q_2 + w_3*Q_3 + w_4*Q_4

    return variance_matrix

variance_matrix = update_the_variance_matrix(residual,Z_STATE,Z_YR)

In [5]:
### GLS procedure

### GLS procedure using linalg.pinv, this works well even you have multicollinearity
def GLS_pseudoinverse_procedure(X,Y,variance_matrix):
    beta_gls = np.linalg.pinv(X.T @ np.linalg.pinv(variance_matrix) @ X) @ X.T @ np.linalg.pinv(variance_matrix) @ Y
    residual_gls = Y - X @ beta_gls

    return beta_gls,residual_gls

### Standard GLS procedure using np.linalg.inv
def GLS_procedure(X,Y,variance_matrix):
    beta_gls = np.linalg.inv(X.T @ np.linalg.inv(variance_matrix) @ X) @ X.T @ np.linalg.inv(variance_matrix) @ Y
    residual_gls = Y - X @ beta_gls

    return beta_gls,residual_gls



In [6]:
## The reuslt of standard GLS procedure, after drop the first term in the Year dummy, the coefficient converges.
for i in range(10):
    beta,residual = GLS_procedure(X=Random_X,Y=Y,variance_matrix=variance_matrix)
    variance_matrix = update_the_variance_matrix(residual=residual,Z_STATE=Z_STATE,Z_YR=Z_YR)
    estimate_u_hat = (residual.T@residual)/(Random_X.shape[0]-Random_X.shape[1]-1)

    print(beta[:4])

[-1.02619933  0.52720064 -0.09843756  4.52655017]
[-1.02562187  0.5274804  -0.09809581  4.52187892]
[-1.02551127  0.52747885 -0.09816123  4.5217926 ]
[-1.02549626  0.5274769  -0.09817426  4.52180663]
[-1.02549436  0.52747658 -0.09817609  4.52180951]
[-1.02549413  0.52747653 -0.09817632  4.52180993]
[-1.0254941   0.52747653 -0.09817635  4.52180997]
[-1.0254941   0.52747653 -0.09817635  4.52180998]
[-1.0254941   0.52747653 -0.09817635  4.52180997]
[-1.0254941   0.52747652 -0.09817635  4.52180998]


In [11]:
## The GLS procedure using linalg.pinv also converges to the same result as the previous GLS procedure
for i in range(10):
    beta,residual = GLS_pseudoinverse_procedure(X=Random_X,Y=Y,variance_matrix=variance_matrix)
    variance_matrix = update_the_variance_matrix(residual=residual,Z_STATE=Z_STATE,Z_YR=Z_YR)
    estimate_u_hat = (residual.T@residual)/(Random_X.shape[0]-Random_X.shape[1]-1)

    print(beta[:4])

[-1.01642193  0.51687575 -0.12936621  1.97174798]
[-1.02972137  0.52742002 -0.0956016   1.90634123]
[-1.02718653  0.52765969 -0.09677998  1.9046346 ]
[-1.02575384  0.52751593 -0.09793718  1.90489467]
[-1.02552722  0.52748235 -0.09814398  1.90500249]
[-1.02549818  0.52747729 -0.09817226  1.90502142]
[-1.0254946   0.52747662 -0.09817585  1.90502403]
[-1.02549416  0.52747654 -0.09817629  1.90502437]
[-1.02549411  0.52747653 -0.09817635  1.90502441]
[-1.0254941   0.52747653 -0.09817635  1.90502441]


In [12]:
### Show the final GLS result
import scipy.stats as stats
beta_variance = np.linalg.inv(Random_X.T @ np.linalg.inv(variance_matrix) @ Random_X)*estimate_u_hat
se = np.sqrt(np.diag(beta_variance))
t_stat = beta/se
pval = 2 * stats.t.cdf(-abs(t_stat),df=Random_X.shape[0]-Random_X.shape[1]-1)

print(pd.DataFrame({'beta':beta[:3],'se':se[:3],'t_stat':t_stat[:3],'pval':pval[:3]},index=['ln_Price','ln_NDI','ln_PIMIN']))

              beta        se      t_stat           pval
ln_Price -1.025494  0.003034 -338.004087   0.000000e+00
ln_NDI    0.527477  0.003373  156.359857   0.000000e+00
ln_PIMIN -0.098176  0.003913  -25.087521  1.344413e-113


(b) Extend the estimation in part (a) by including correlated random effects for 𝜇𝑖 using the  time averages of ln 𝑃rice𝑖t, ln 𝑁DI𝑖t, and ln 𝑃IMIN𝑖t as additional regressors.

In [13]:
df['region_ln_price'] = df.groupby('STATE')['ln_Price'].transform('mean')
df['region_ln_NDI'] = df.groupby('STATE')['ln_NDI'].transform('mean')
df['region_ln_PIMIN'] = df.groupby('STATE')['ln_PIMIN'].transform('mean')

X = df[['ln_Price','ln_NDI','ln_PIMIN','region_ln_price','region_ln_NDI','region_ln_PIMIN']]
Random_X = pd.concat([X,Z_YR],axis=1)
Random_X = np.array(Random_X,dtype=float)

beta = np.linalg.inv(Random_X.T @ Random_X) @ Random_X.T @ Y
residual = Y - Random_X @ beta

for i in range(10):
    beta,residual = GLS_pseudoinverse_procedure(X=Random_X,Y=Y,variance_matrix=variance_matrix)
    variance_matrix = update_the_variance_matrix(residual=residual,Z_STATE=Z_STATE,Z_YR=Z_YR)
    estimate_u_hat = (residual.T@residual)/(Random_X.shape[0]-Random_X.shape[1]-1)

    print(beta[:6])

[-1.02446729  0.52331137 -0.10558725 -0.41096574]
[-1.01900125  0.51697335 -0.12342188 -0.46560558]
[-1.01844049  0.51634015 -0.12515941 -0.47066404]
[-1.01838295  0.51627791 -0.12532612 -0.47110746]
[-1.01837687  0.5162716  -0.12534262 -0.47114698]
[-1.01837621  0.51627094 -0.12534431 -0.4711506 ]
[-1.01837614  0.51627087 -0.12534449 -0.47115095]
[-1.01837613  0.51627086 -0.12534451 -0.47115098]
[-1.01837613  0.51627086 -0.12534451 -0.47115098]
[-1.01837613  0.51627086 -0.12534451 -0.47115098]


In [14]:
import scipy.stats as stats
beta_variance = np.linalg.inv(Random_X.T @ np.linalg.inv(variance_matrix) @ Random_X)*estimate_u_hat
se = np.sqrt(np.diag(beta_variance))
t_stat = beta/se
pval = 2 * stats.t.cdf(-abs(t_stat),df=Random_X.shape[0]-Random_X.shape[1]-1)

print(pd.DataFrame({'beta':beta[:6],'se':se[:6],'t_stat':t_stat[:6],'pval':pval[:6]},index=['ln_Price','ln_NDI','ln_PIMIN','region_ln_price','region_ln_NDI','region_ln_PIMIN']))

                     beta        se     t_stat           pval
ln_Price        -1.018376  0.015742 -64.690801   0.000000e+00
ln_NDI           0.516271  0.017637  29.272280  4.226539e-146
ln_PIMIN        -0.125345  0.020429  -6.135498   1.114999e-09
region_ln_price -0.471151  0.020607 -22.863214   5.519484e-98
region_ln_NDI    0.089092  0.018465   4.824813   1.561444e-06
region_ln_PIMIN  0.447316  0.023642  18.920719   5.770675e-71


(c) Further extend the estimation by including the three regressors Pop, Pop16 and CPI in the model. Compare the results with those in part (b).

In [15]:
X = df[['ln_Price','ln_NDI','ln_PIMIN','region_ln_price','region_ln_NDI','region_ln_PIMIN','Pop','Pop16','CPI']]
Random_X = pd.concat([X,Z_YR],axis=1)
Random_X = np.array(Random_X,dtype=float)

beta = np.linalg.inv(Random_X.T @ Random_X) @ Random_X.T @ Y
residual = Y - Random_X @ beta

for i in range(10):
    beta,residual = GLS_pseudoinverse_procedure(X=Random_X,Y=Y,variance_matrix=variance_matrix)
    variance_matrix = update_the_variance_matrix(residual=residual,Z_STATE=Z_STATE,Z_YR=Z_YR)
    estimate_u_hat = (residual.T@residual)/(Random_X.shape[0]-Random_X.shape[1]-1)

    print(beta[:9])

[-1.01628644  0.52344955 -0.1257655  -0.40813067]
[-1.01772732  0.52682486 -0.11633606 -0.37437203]
[-1.01844142  0.52824331 -0.11238419 -0.36177168]
[-1.01876262  0.52885157 -0.11068947 -0.35658888]
[-1.01890413  0.52911538 -0.10995437 -0.35437681]
[-1.01896628  0.5292305  -0.10963344 -0.35341788]
[-1.01899351  0.52928087 -0.10949313 -0.35299941]
[-1.01900545  0.52930289 -0.1094316  -0.35281622]
[-1.01901069  0.52931255 -0.10940464 -0.35273595]
[-1.01901296  0.52931683 -0.10939282 -0.3527008 ]


In [16]:
import scipy.stats as stats
beta_variance = np.linalg.inv(Random_X.T @ np.linalg.inv(variance_matrix) @ Random_X)*estimate_u_hat
se = np.sqrt(np.diag(beta_variance))
t_stat = beta/se
pval = 2 * stats.t.cdf(-abs(t_stat),df=Random_X.shape[0]-Random_X.shape[1]-1)

print(pd.DataFrame({'beta':beta[:9],'se':se[:9],'t_stat':t_stat[:9],'pval':pval[:9]},index=['ln_Price','ln_NDI','ln_PIMIN','region_ln_price','region_ln_NDI','region_ln_PIMIN','Pop','Pop16','CPI']))

                     beta          se     t_stat           pval
ln_Price        -1.019013    0.015504 -65.723738   0.000000e+00
ln_NDI           0.529317    0.017247  30.689531  4.347136e-157
ln_PIMIN        -0.109393    0.020004  -5.468655   5.403537e-08
region_ln_price -0.352701    0.020331 -17.348145   5.624227e-61
region_ln_NDI    0.127952    0.018180   7.037918   3.105761e-12
region_ln_PIMIN  0.395541    0.023265  17.001839   7.485307e-59
Pop             -0.000023    0.000003  -7.962938   3.557362e-15
Pop16            0.000023    0.000004   5.852377   6.084912e-09
CPI              0.039779  482.007474   0.000083   9.999342e-01


We can found that there is only little change on the coefficient and their statistical significance do not change so much

(d) Estimate the model in part (a) by treating 𝜇𝑖 and 𝜆𝑡 as fixed effects. Compare the results with those in part (a).

| Variable |      Beta |       SE |     T-stat |           P-value |
|----------|----------:|---------:|-----------:|------------------:|
| ln_Price | -1.025494 | 0.003034 | -338.004087|       0.000000e+00 |
| ln_NDI   |  0.527477 | 0.003373 |  156.359857|       0.000000e+00 |
| ln_PIMIN | -0.098176 | 0.003913 |  -25.087521| 1.344413e-113      |


In [29]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

data_combined = df.set_index(['STATE', 'YR'])
data_combined['ln_C'] = np.log(data_combined['C'])
exog_vars = ['ln_Price','ln_NDI','ln_PIMIN']
exog = sm.add_constant(data_combined[exog_vars])
mod = PanelOLS(data_combined['ln_C'], exog, time_effects=True,entity_effects=True)
panel_res = mod.fit()
print(panel_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   ln_C   R-squared:                        0.3966
Estimator:                   PanelOLS   R-squared (Between):              0.3139
No. Observations:                1380   R-squared (Within):              -3.3929
Date:                Wed, Mar 06 2024   R-squared (Overall):             -0.8845
Time:                        16:36:53   Log-likelihood                    1664.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      285.26
Entities:                          46   P-value                           0.0000
Avg Obs:                       30.000   Distribution:                  F(3,1302)
Min Obs:                       30.000                                           
Max Obs:                       30.000   F-statistic (robust):             285.26
                            

The result is almost the same, while there is some small change on the std error.

#### 2. Consider the estimation of the two-way random effects model in Problem 1 above again, but using the maximum likelihood approach. The partially maximized loglikelihood function of 𝜙2 and 𝜙3 takes the form:....

(a) Explore and summarize how Python module scipy can help maximizing log 𝐿𝑐(𝜙2, 𝜙32). You may like to refer to the SciPy Documentation available at https://docs.scipy.org.



(1). Get the parameter you use to minimize the likehood function

(2). Express other variable into the variable of your parameter

(3). Write down your likehood function

(4). Choose a begin value of your parameter and range of your parameter

(5). Minimize your likehood function


(b) Write a Python script to implement the maximization of log 𝐿𝑐(𝜙2, 𝜙3), or the minimization of −log 𝐿𝑐(𝜙22, 𝜙32), based on the cigarette demand data: cigar.xlsx

In [1]:
import numpy as np
import pandas as pd


### Prepare : Get X and Y
df = pd.read_excel('cigar.xlsx')
df.columns = df.columns.str.strip()
df = df.dropna()
Z_STATE = pd.get_dummies(df['STATE'])
Z_YR = pd.get_dummies(df['YR'])

Z_STATE_1 = pd.get_dummies(df['STATE'],drop_first=True)

Y = np.log(df['C'])
df['ln_Price'] = np.log(df['Price'])
df['ln_NDI'] = np.log(df['NDI'])
df['ln_PIMIN'] = np.log(df['PIMIN'])
X = df[['ln_Price','ln_NDI','ln_PIMIN']]

X = pd.concat([X,Z_STATE_1,Z_YR],axis=1)
X = np.array(X,dtype=float)
Y = np.array(Y,dtype=float)


In [2]:
### Find out your parameter: phi_2; phi_3
### Find out how to express the omega using phi_2 and phi_3
### omega = w_1*Q_1 + w_2*Q_2 + w_3*Q_3 + w_4*Q_4

## So we need to find out the Q matrix first as before


J_n = np.ones(Z_STATE.shape[1]).reshape(Z_STATE.shape[1],1)@np.ones(Z_STATE.shape[1]).reshape(Z_STATE.shape[1],1).T
J_t = np.ones(Z_YR.shape[1]).reshape(Z_YR.shape[1],1)@np.ones(Z_YR.shape[1]).reshape(Z_YR.shape[1],1).T


bar_J_n = (1/(Z_STATE.shape[1])) * J_n
bar_J_t = (1/(Z_STATE.shape[1])) * J_t


E_n = np.eye(Z_STATE.shape[1]) - bar_J_n
E_t = np.eye(Z_YR.shape[1]) - bar_J_t

## estimate the Q matrix
Q_1 = np.kron(E_n,E_t)
Q_2 = np.kron(E_n,bar_J_t)
Q_3 = np.kron(bar_J_n,E_t)
Q_4 = np.kron(bar_J_n,bar_J_t)

In [7]:
### Then express the w_i

### w_2 = sigma_v^2/phi_2
### w_3 = sigma_v^2/phi_3
### w_1 = sigma_v^2
### w_4 = w_2+w_3-w_1 = sigma_v^2/phi_2 + sigma_v^2/phi_3 - sigma_v^2
### omega = w_1*Q_1 + w_2*Q_2 + w_3*Q_3 + w_4*Q_4 = sigma_v^2(Q1+1/phi_2*Q2+1/phi_3*Q3+(1/phi_2+1/phi_3-1)Q4)
### Large_sigma = sigma_v^-2*omega = Q1+1/phi_2*Q2+1/phi_3*Q3+(1/phi_2+1/phi_3-1)Q4

### Then we begin express the likelihood function
from scipy.optimize import minimize


## We define the compute metrice function in order to express some variables using phi_2 and phi_3
def compute_metrice(Q_1,Q_2,Q_3,Q_4,phi_2,phi_3,X,Y,n,t):
    large_sigma = Q_1 + (1/phi_2)*Q_2 + (1/phi_3)*Q_3 + (1/phi_2+1/phi_3-1)*Q_4
    large_sigma_inv = Q_1 + phi_2*Q_2 + phi_3*Q_3 + (phi_2*phi_3)/(phi_2+phi_3-(phi_2*phi_3))*Q_4
    beta = np.linalg.inv(X.T @ large_sigma_inv @ X) @ X.T @ large_sigma_inv @ Y
    mu = Y-X@beta
    sigma_v = (1/n*t)*(Y-X@beta).T@large_sigma_inv@(Y-X@beta)

    return beta,mu,sigma_v,large_sigma_inv


## Then we define the likelihood function

def neg_log_likelihood(params,Q_1, Q_2, Q_3, Q_4, Y, X,n,t):
    phi_2 = params[0]
    phi_3 = params[1]


    beta,mu,sigma_v,large_sigma_inv = compute_metrice(Q_1=Q_1,
                                          Q_2=Q_2,
                                          Q_3=Q_3,
                                          Q_4=Q_4,
                                          phi_2=phi_2,
                                          phi_3=phi_3,
                                          X =X,
                                          Y=Y,
                                          n=n,
                                          t=t)
    ## constant will not change the reuslt, ignore it
    term_1 = -((n*t)/2)*np.log(sigma_v)
    term_2 = (n/2)*np.log(phi_2)
    term_3 = (t/2)*np.log(phi_3)
    term_4 = (-1/2)*np.log(phi_2+phi_3-(phi_2*phi_3))
    term_5 = -(1/(2*sigma_v))*mu.T@large_sigma_inv@mu



    nll = -(term_1+term_2+term_3+term_4+term_5)

    return nll

## In pratice, we find sometime term4 will be negative, so we need to add a constraint to make sure term4 is positive
def constraint(params):
    phi_2, phi_3 = params
    return phi_2 + phi_3 - phi_2 * phi_3

cons = ({'type': 'ineq', 'fun': constraint})



### set the intial_params and bounds
initial_params = [0.1,0.1]
bounds = [(0.0001, None)]*2
n= Z_STATE.shape[1]
t=Z_YR.shape[1]
## 'TNC', 'SLSQP', 'Nelder-Mead' can be used here, but we find the SLSQP is the best
method = 'SLSQP'
### optimize the function
result = minimize(neg_log_likelihood, initial_params, args=(Q_1, Q_2, Q_3, Q_4, Y, X,n,t),method=method, bounds=bounds,constraints=cons)

if result.success:
    fitted_params = result.x
    print('Optimal parameters:', fitted_params)
else:
    raise ValueError(result.message)

     

C:\Users\Kororinpa\AppData\Local\Temp\ipykernel_4296\1082080352.py:44: RuntimeWarning: invalid value encountered in log
  term_4 = (-1/2)*np.log(phi_2+phi_3-(phi_2*phi_3))


Optimal parameters: [2.26194218 1.79240975]


In [9]:
### After getting the result, we can use the fitted_params to compute the beta, mu, sigma_v, large_sigma_inv
beta,mu,sigma_v,large_sigma_inv = compute_metrice(Q_1=Q_1,Q_2=Q_2,Q_3=Q_3,Q_4=Q_4,phi_2=fitted_params[0],phi_3=fitted_params[0],X=X,Y=Y,n=n,t=t)

| Variable |      Beta |       SE |     T-stat |           P-value |
|----------|----------:|---------:|-----------:|------------------:|
| ln_Price | -1.025494 | 0.003034 | -338.004087|       0.000000e+00 |
| ln_NDI   |  0.527477 | 0.003373 |  156.359857|       0.000000e+00 |
| ln_PIMIN | -0.098176 | 0.003913 |  -25.087521| 1.344413e-113      |

In [12]:
beta[:3]

array([-1.02457955,  0.52564843, -0.10304916])

We show the parameters of ln_price, ln_ndi and ln_pimin here. After comparing it with the GLS results, we find the coefficient of them are really similar, which proves that this MLE procedure was a successful one.